# 🍏 Health Resource Search Agent Tutorial 🍎

Welcome to the **Health Resource Search Agent** tutorial! We'll use **Azure AI Foundry** SDKs to build an assistant that can:

1. **Upload** health and recipe files into a vector store.
2. **Create an Agent** with a **File Search** tool.
3. **Search** these documents for relevant dietary info.
4. **Answer** health and wellness questions (with disclaimers!).

### ⚠️ Important Medical Disclaimer ⚠️
> **All health information in this notebook is for general educational purposes only and is not a substitute for professional medical advice, diagnosis, or treatment.** Always seek the advice of a qualified healthcare professional with any questions you may have.

## Prerequisites
- Complete Agent basics notebook - [1-basics.ipynb](1-basics.ipynb)
- **Roles**  
  1. **Azure AI Developer** on your Azure AI Foundry project.
  2. **Storage Blob Data Contributor** on the project’s Storage account.
  3. If standard agent setup is used with your own Search resource, also ensure you have **Cognitive Search Data Contributor** on that resource.

## Let's Get Searching!
We'll show you how to upload some sample files, create a vector store for them, then spin up an agent that can search these resources for dietary guidelines, recipes, and more. Enjoy!

<img src="./seq-diagrams/3-file-search.png" width="30%"/>


## 1. Initial Setup
Here we import needed libraries, load environment variables from `.env`, and initialize our **AIProjectClient**. Let's do this! 🎉

In [ ]:
import os
import time
from pathlib import Path

from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import (
    FileSearchTool,
    FilePurpose,
    MessageTextContent,
    MessageRole
)

# Load environment variables from parent .env
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir.parent / '.env')

# Initialize AIProjectClient
try:
  project_client = AIProjectClient(
      credential=DefaultAzureCredential(),
      endpoint=os.environ.get("PROJECT_CONNECTION_STRING")
  )
  print("✅ Successfully initialized AIProjectClient")
except Exception as e:
  print(f"❌ Error initializing project client: {e}")

✅ Successfully initialized AIProjectClient


## 2. Prepare Sample Files 🍲🗒
We'll create some dummy .md files (for recipes and guidelines). Then we'll store them in a vector store for searching.


In [ ]:
def create_sample_files():
  recipes_md = (
      """# Healthy Recipes Database\n\n"
        "## Gluten-Free Recipes\n"
        "1. Quinoa Bowl\n"
        "   - Ingredients: quinoa, vegetables, olive oil\n"
        "   - Instructions: Cook quinoa, add vegetables\n\n"
        "2. Rice Pasta with Vegetables\n"
        "   - Ingredients: rice pasta, mixed vegetables\n"
        "   - Instructions: Boil pasta, sauté vegetables\n\n"
        "## Diabetic-Friendly Recipes\n"
        "1. Low-Carb Stir Fry\n"
        "   - Ingredients: chicken, vegetables, tamari sauce\n"
        "   - Instructions: Cook chicken, add vegetables\n\n"
        "2. Greek Salad\n"
        "   - Ingredients: cucumber, tomatoes, feta, olives\n"
        "   - Instructions: Chop vegetables, combine\n\n"
        "## Heart-Healthy Recipes\n"
        "1. Baked Salmon\n"
        "   - Ingredients: salmon, lemon, herbs\n"
        "   - Instructions: Season salmon, bake\n\n"
        "2. Mediterranean Bowl\n"
        "   - Ingredients: chickpeas, vegetables, tahini\n"
        "   - Instructions: Combine ingredients\n"""
  )

  guidelines_md = (
      """# Dietary Guidelines\n\n"
        "## General Guidelines\n"
        "- Eat a variety of foods\n"
        "- Control portion sizes\n"
        "- Stay hydrated\n\n"
        "## Special Diets\n"
        "1. Gluten-Free Diet\n"
        "   - Avoid wheat, barley, rye\n"
        "   - Focus on naturally gluten-free foods\n\n"
        "2. Diabetic Diet\n"
        "   - Monitor carbohydrate intake\n"
        "   - Choose low glycemic foods\n\n"
        "3. Heart-Healthy Diet\n"
        "   - Limit saturated fats\n"
        "   - Choose lean proteins\n"""
  )

  # Save to local .md files
  with open("recipes.md", "w", encoding="utf-8") as f:
    f.write(recipes_md)
  with open("guidelines.md", "w", encoding="utf-8") as f:
    f.write(guidelines_md)

  print("📄 Created sample resource files: recipes.md, guidelines.md")
  return ["recipes.md", "guidelines.md"]


sample_files = create_sample_files()

📄 Created sample resource files: recipes.md, guidelines.md


#### ✨ Note on Search Permissions
When creating the vector store, you must also have **Cognitive Search Data Contributor** role on your Azure AI Search resource (if you're using the standard agent setup with your own Search resource). Missing this role will often cause a **Forbidden** error. See [Authentication Setup](../../1-introduction/1-authentication.ipynb#4-add-agent-service-permissions) for details on configuring permissions.


## 3. Create a Vector Store 📚
We'll upload our newly created files and group them into a single vector store for searching. This is how the agent can later find relevant text.

In [ ]:
def create_vector_store(files, store_name="my_health_resources"):
  try:
    # Step 1: Upload files to Azure AI Agent service
    # Each file needs to be uploaded individually and we'll collect their IDs
    uploaded_ids = []
    for fp in files:
      # upload_file_and_poll ensures the upload is complete before continuing
      # FilePurpose.AGENTS tells the service these files are for agent usage
      upl = project_client.agents.files.upload_and_poll(
          file_path=fp,
          purpose=FilePurpose.AGENTS
      )
      uploaded_ids.append(upl.id)
      print(f"✅ Uploaded: {fp} -> File ID: {upl.id}")

    # Step 2: Create a vector store from the uploaded files
    # A vector store converts text into numerical vectors for semantic search
    # create_vector_store_and_poll waits until indexing is complete
    vs = project_client.agents.vector_stores.create_and_poll(
        file_ids=uploaded_ids,  # Pass all our uploaded file IDs
        name=store_name         # Give our vector store a friendly name
    )
    print(f"🎉 Created vector store '{store_name}', ID: {vs.id}")
    return vs, uploaded_ids
  except Exception as e:
    print(f"❌ Error creating vector store: {e}")
    return None, []


# Initialize empty variables to store our vector store and file IDs
vector_store, file_ids = None, []

# If we successfully created sample files earlier, create a vector store from them
if sample_files:
  vector_store, file_ids = create_vector_store(
      sample_files, "health_resources_example")

✅ Uploaded: recipes.md -> File ID: assistant-76bKJV77wmSMHpmyyqx1Jx
✅ Uploaded: guidelines.md -> File ID: assistant-3dL9Wq83jCxVTbTrcBU5Vp
🎉 Created vector store 'health_resources_example', ID: vs_iedO9EMvU5BhFvIOuzvHPxA7


## 4. Create the Health Resource Agent 🔎
We use a **FileSearchTool** pointing to our newly created vector store, then create the Agent with instructions about disclaimers, dietary help, etc.

In [5]:
def create_health_resource_agent(vstore_id):
  try:
    # Create a FileSearchTool that will allow our agent to search through documents
    # FileSearchTool uses the vector store we created earlier to perform semantic search
    # This means the agent can find relevant content even if the exact words don't match
    file_search_tool = FileSearchTool(vector_store_ids=[vstore_id])

    # Create an AI agent that will use our search tool and follow specific instructions
    # The agent combines:
    # 1. A base LLM model (specified in environment variables or defaults to gpt-4o-mini)
    # 2. The ability to search our health documents using the FileSearchTool
    # 3. Custom instructions that shape how it responds to questions
    agent = project_client.agents.create_agent(
        # Specify which LLM model to use - fallback to gpt-4o-mini if not set
        model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o-mini"),

        # Give our agent a descriptive name
        name="health-search-agent",

        # These instructions act like a personality and rule set for our agent
        # They ensure consistent, responsible health advice
        instructions="""
                You are a health resource advisor with access to dietary and recipe files.
                You:
                1. Always present disclaimers (you're not a doctor!)
                2. Provide references to the files when possible
                3. Focus on general nutrition or recipe tips.
                4. Encourage professional consultation for more detailed advice.
            """,

        # Connect the search tool's interface definition and required resources
        # This gives the agent the ability to actually perform searches
        tools=file_search_tool.definitions,
        tool_resources=file_search_tool.resources
    )
    print(f"🎉 Created health resource agent, ID: {agent.id}")
    return agent
  except Exception as e:
    print(f"❌ Error creating health resource agent: {e}")
    return None


# Initialize our agent variable
health_agent = None

# Only create the agent if we successfully created a vector store earlier
if vector_store:
  health_agent = create_health_resource_agent(vector_store.id)

🎉 Created health resource agent, ID: asst_Uv1BGGU3yTS1fqjefYJeKdlo


## 5. Searching Health Resources 🏋️👩‍🍳
We'll create a new conversation thread and ask queries like “Gluten-free recipe ideas?” or “Heart-healthy meal plan?” The agent will do file search on the vector store to find relevant info.

In [6]:
def create_search_thread(agent):
  try:
    # In Azure AI Agent service, conversations happen in "threads"
    # A thread maintains the context and history of a conversation
    # Here we create a new empty thread to start a fresh conversation
    thread = project_client.agents.threads.create()
    print(f"📝 Created new search thread, ID: {thread.id}")
    return thread
  except Exception as e:
    print(f"❌ Error creating search thread: {e}")
    return None


def ask_search_question(thread_id, agent_id, user_question):
  try:
    # First, we add the user's question as a message to the thread
    # This is like typing a message in a chat interface
    message = project_client.agents.messages.create(
        thread_id=thread_id,
        role="user",  # Specifies this message is from the user
        content=user_question
    )
    print(f"🔎 Searching: '{user_question}'")

    # Next, we create and process a "run" - this is where the magic happens!
    # The agent will:
    # 1. Read the user's question
    # 2. Use the FileSearchTool to search our health documents
    # 3. Generate a helpful response based on the search results
    run = project_client.agents.runs.create_and_process(
        thread_id=thread_id,
        agent_id=agent_id
    )
    print(f"🤖 Run finished with status: {run.status}")
    if run.last_error:
      print(f"Error details: {run.last_error}")
    return run
  except Exception as e:
    print(f"❌ Error searching question: {e}")
    return None


# Now let's test our search functionality!
# First check if we have our health agent available
if health_agent:
  # Create a new conversation thread
  search_thread = create_search_thread(health_agent)

  if search_thread:
    # Define some test questions that demonstrate different types of health queries
    # The agent will search our uploaded health documents to answer these
    queries = [
        "Could you suggest a gluten-free lunch recipe?",
        "Show me some heart-healthy meal ideas.",
        "What guidelines do you have for someone with diabetes?"
    ]

    # Process each query one at a time
    # The agent will maintain conversation context between questions
    for q in queries:
      ask_search_question(search_thread.id, health_agent.id, q)

📝 Created new search thread, ID: thread_G1FEQmVvSoc9wjLbvJqO0rkr
🔎 Searching: 'Could you suggest a gluten-free lunch recipe?'
🤖 Run finished with status: RunStatus.COMPLETED
🔎 Searching: 'Show me some heart-healthy meal ideas.'
🤖 Run finished with status: RunStatus.COMPLETED
🔎 Searching: 'What guidelines do you have for someone with diabetes?'
🤖 Run finished with status: RunStatus.COMPLETED


## 6. View Results & Citations 📄
We'll read the conversation thread to see how the agent responded and see if it cited the correct files.

In [ ]:
def display_thread_messages(thread_id):
  try:
    # Retrieve all messages in this conversation thread using the Azure AI Agent SDK
    # Messages contain the back-and-forth between user and AI agent
    messages = list(project_client.agents.messages.list(thread_id=thread_id))

    # Display the conversation history in reverse chronological order (newest first)
    print("\n🗣️ Conversation so far:")
    for m in reversed(messages):
      # Each message may have multiple content pieces
      # We're interested in the text content (vs other types like images)
      if m.text_messages:
        last_content = m.text_messages[-1]
        if hasattr(last_content, "text"):
          # Print who said what (ASSISTANT or USER) along with their message
          print(f"{m.role.upper()}: {last_content.text.value}\n")

      # The agent can cite specific passages from the uploaded documents
      # Let's check if it referenced any files in its responses
      if m.file_citation_annotations:
        # If the message has file citations, we can display them
        print("\n📎 Checking for citations...")
        for c in m.file_citation_annotations:
          # Each citation includes the quoted text and which file it came from
          # This helps users verify the agent's sources
          print(
              f"- Citation snippet: '{c.text}' from file ID: {c.file_citation['file_id']}")

  except Exception as e:
    # Gracefully handle any errors that might occur when displaying messages
    print(f"❌ Error displaying messages: {e}")


# Display the conversation history for our search thread
if search_thread:
  display_thread_messages(search_thread.id)


🗣️ Conversation so far:
USER: Could you suggest a gluten-free lunch recipe?

ASSISTANT: Certainly! Here's a simple and delicious gluten-free lunch idea: Quinoa Salad with Vegetables and Feta.

**Ingredients:**
- 1 cup cooked quinoa (a gluten-free grain)
- Cherry tomatoes, halved
- Cucumber, diced
- Red onion, finely chopped
- Feta cheese, crumbled
- Fresh parsley, chopped
- Olive oil
- Lemon juice
- Salt and pepper to taste

**Instructions:**
1. Cook the quinoa according to package instructions and let it cool.
2. In a large bowl, combine the cooled quinoa, cherry tomatoes, cucumber, red onion, and feta cheese.
3. Drizzle with olive oil and lemon juice.
4. Season with salt and pepper, then toss gently.
5. Garnish with chopped parsley before serving.

This recipe is naturally gluten-free, nutritious, and easy to prepare. 

**Disclaimer:** While this recipe is gluten-free, always ensure all ingredients, especially processed items like feta and olive oil, are certified gluten-free if you

## 7. Cleanup & Best Practices 🧹
We'll optionally remove the vector store, the uploaded files, and the agent. In a production environment, you might keep them around longer. Meanwhile, here are some tips:

1. **Resource Management**
   - Keep files grouped by category, regularly prune old or irrelevant files.
   - Clear out test agents or vector stores once you're done.

2. **Search Queries**
   - Provide precise or multi-part queries.
   - Consider synonyms or alternative keywords ("gluten-free" vs "celiac").
   
3. **Health Information**
   - Always disclaim that you are not a medical professional.
   - Encourage users to see doctors for specific diagnoses.

4. **Performance**
   - Keep an eye on vector store size.
   - Evaluate search accuracy with `azure-ai-evaluation`!


In [10]:
def cleanup_all():
  try:
    # Check if we have a vector store and delete it
    # Vector stores are where we store the embeddings (numerical representations)
    # of our documents for semantic search
    if 'vector_store' in globals() and vector_store:
      project_client.agents.vector_stores.delete(vector_store.id)
      print("🗑️ Deleted vector store.")

    # Remove any files we uploaded to Azure AI Search
    # These were the documents our agent used as its knowledge base
    if 'file_ids' in globals() and file_ids:
      for fid in file_ids:
        project_client.agents.files.delete(fid)
      print("🗑️ Deleted uploaded files from the service.")

    # Delete the AI agent we created
    # This frees up resources since we're done with our demo
    if 'health_agent' in globals() and health_agent:
      project_client.agents.delete_agent(health_agent.id)
      print("🗑️ Deleted health resource agent.")

    # Clean up any local files we created during the demo
    # This keeps our workspace tidy
    if 'sample_files' in globals() and sample_files:
      for sf in sample_files:
        if os.path.exists(sf):
          os.remove(sf)
      print("🗑️ Deleted local sample files.")

  except Exception as e:
    # If anything goes wrong during cleanup, we'll see what happened
    print(f"❌ Error during cleanup: {e}")


# Run our cleanup function to remove all resources we created
# This is good practice in a tutorial/demo environment
cleanup_all()

🗑️ Deleted vector store.
🗑️ Deleted uploaded files from the service.
🗑️ Deleted health resource agent.
🗑️ Deleted local sample files.


# Congratulations! 🎉
You've created a **Health Resource Search Agent** that:
1. Uses a **Vector Store** to store sample recipes & guidelines.
2. **Searches** them to answer queries.
3. **Provides disclaimers** reminding users to consult professionals.

Feel free to adapt this approach for your own corporate documents, product manuals, or custom health resources.

Happy Searching! 🎉

#### Let's proceed to [4-bing_grounding.ipynb](4-bing_grounding.ipynb)